In [ ]:
!pip install txtai autoawq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver 

In [ ]:
from txtai.pipeline import LLM
from txtai import Embeddings

mistral = LLM("TheBloke/Mistral-7B-OpenOrca-AWQ")

def get_answer(question, context):
  prompt = f"""<|im_start|>system
  You are a Q&A assistant. You give answers that are only based on the provided context. Your answers contain only what is asked and nothing more.
  Context: {context}<|im_end|>
  <|im_start|>user
  Answer the following question.
  Question: {question}<|im_end|>
  Answer:<|im_start|>assistant

  """
  return mistral(prompt, maxlength=1024)

wiki_embeddings = Embeddings()
wiki_embeddings.load(provider="huggingface-hub", container="neuml/txtai-wikipedia")

def get_context(question):
  context = "\n".join([x["text"] for x in wiki_embeddings.search(question)])
  return context

def rag(question):
  return get_answer(question, get_context(question))

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Device set to use cuda:0


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import json

def extract_model_input(jsonl_file_path):
    model_inputs = []
    try:
        with open(jsonl_file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Parse each line as JSON
                data = json.loads(line)
                # Check if 'model_input' exists in the JSON object
                if 'model_input' in data:
                    model_inputs.append(data['model_input'])
    except FileNotFoundError:
        print(f"Error: File not found at {jsonl_file_path}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    return model_inputs

def extract_model_output(jsonl_file_path):
    model_outputs = []
    try:
        with open(jsonl_file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Parse each line as JSON
                data = json.loads(line)
                # Check if 'model_output_text' exists in the JSON object
                if 'model_output_text' in data:
                    model_outputs.append(data['model_output_text'])
    except FileNotFoundError:
        print(f"Error: File not found at {jsonl_file_path}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    return model_outputs

if __name__ == "__main__":
    input_file_path = "mushroom.en-train_nolabel.v1.jsonl"
    model_inputs_list = extract_model_input(input_file_path)
    model_outputs_list = extract_model_output(input_file_path)



In [ ]:
answers = []
for question in model_inputs_list:
  answers.append(rag(question))

In [ ]:
with open("rag_answers.txt", "w") as f:
  for answer in answers:
    f.write(answer + "\n")

In [ ]:
#No need for this one if you're using model_output_text
def get_hallucination(question, answer):
  prompt = f"""<|im_start|>system
  You are a data augmentation system.
  You are given a question and a true answer and you produce another answer.
  You answer the question by making minimal changes to the true answer.
  Keep the same number of tokens and align the sentences.<|im_end|>
  <|im_start|>user
  Question: {question}
  Sentence: {answer}
  Generate another answer to the given question. Make a different claim.<|im_end|>
  Answer:<|im_start|>assistant

  """
  return mistral(prompt, maxlength=1024)

hallucinated_answers = []
for question, answer in zip(model_inputs_list, answers):
  hallucinated_answers.append(get_hallucination(question, answer))

with open("hallucinated_answers.txt", "w") as f:
  for answer in hallucinated_answers:
    f.write(answer + "\n")

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.7.1/en_core_web_lg-3.7.1-py3-none-any.whl (587.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install rake-nltk

In [ ]:
import re
import spacy
import pandas as pd
import nltk
from collections import Counter
from rake_nltk import Rake
from sentence_transformers import SentenceTransformer, util

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt_tab')

# Load spaCy model
nlp = spacy.load("en_core_web_lg")

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize RAKE
rake = Rake()

def extract_named_entities(text):
    """Extract named entities using spaCy."""
    doc = nlp(text)
    named_entities = {ent.text for ent in doc.ents}
    return named_entities

def extract_keywords(text):
    """Extract keywords using RAKE."""
    rake.extract_keywords_from_text(text)
    keywords = rake.get_ranked_phrases()
    return keywords

def split_keywords_into_words(keywords):
    """Split keywords into individual words."""
    words = set()
    for keyword in keywords:
        words.update(keyword.split())
    return words

def split_into_clauses(text):
    """Split text into clauses based on syntactic dependencies using spaCy, ensuring proper spacing."""
    doc = nlp(text)
    clauses = []
    for sentence in doc.sents:
        for token in sentence:
            if token.dep_ in {"advcl", "ccomp", "xcomp", "acl", "relcl"} or token == sentence.root:
                clause_text = " ".join([t.text for t in token.subtree])
                clause_text = re.compile(r"\s([,;.:?!])", re.UNICODE).sub(r"\1", clause_text)  # Fix spaces before punctuation
                clauses.append(clause_text)
    return clauses

def find_best_matching_clause(generated_clause, corrected_clauses):
    """Find the best matching clause in corrected_clauses for a generated_clause using Sentence-BERT."""
    generated_embedding = model.encode(generated_clause, convert_to_tensor=True)
    corrected_embeddings = model.encode(corrected_clauses, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(generated_embedding, corrected_embeddings)
    best_match_idx = similarity_scores.argmax().item()
    best_match_clause = corrected_clauses[best_match_idx]
    best_similarity = similarity_scores[0, best_match_idx].item()
    return best_match_clause, best_similarity

def find_hallucinated_clauses(generated_text, corrected_text):
    """Identify hallucinated clauses where all named entities and keywords are hallucinated, or similarity is below threshold."""
    generated_clauses = split_into_clauses(generated_text)
    corrected_clauses = split_into_clauses(corrected_text)

    hallucinated_clauses = []
    clause_similarity = []
    similarity_threshold = 0.55
    for g_clause in generated_clauses:
        c_clause, similarity = find_best_matching_clause(g_clause, corrected_clauses)
        generated_keywords = split_keywords_into_words(extract_keywords(g_clause))
        corrected_keywords = split_keywords_into_words(extract_keywords(c_clause))

        # Check for common keywords
        common_keywords = generated_keywords.intersection(corrected_keywords)
        if len(generated_keywords) == 0:
            raito_common_keywords = 0
        else:
            raito_common_keywords = len(common_keywords) / len(generated_keywords)


        if  (raito_common_keywords <= 0.33) and similarity <= similarity_threshold:
            hallucinated_clauses.append(g_clause)

        # Add similarity score
        clause_similarity.append(similarity)

    # Filter out subsets and keep the largest clause
    filtered_clauses = []
    hallucinated_clauses_sorted = sorted(hallucinated_clauses, key=len, reverse=True)
    for clause in hallucinated_clauses_sorted:
        if not any(clause != other and clause in other for other in hallucinated_clauses_sorted):
            filtered_clauses.append(clause)

    return filtered_clauses, clause_similarity

def update_generated_text(generated_text, hallucinated_clauses):
    """Update the generated text by replacing hallucinated clauses with an empty string and applying strip."""
    for clause in hallucinated_clauses:
        clause = " ".join(clause.split()).replace(" ,", ",").replace(" .", ".")  # Fix spaces before punctuation
        generated_text = generated_text.replace(clause, "")
    return " ".join(generated_text.split()).strip()

def extract_hallucinated_words(generated_text, corrected_text):
    """Extract hallucinated words by comparing generated and corrected texts."""
    generated_keywords = split_keywords_into_words(extract_keywords(generated_text))
    corrected_keywords = split_keywords_into_words(extract_keywords(corrected_text))
    hallucinated_words = generated_keywords - corrected_keywords
    return hallucinated_words

def get_hard_labels(generated_text, hallucinated_words, hallucinated_clauses):
    """Get start and end indices of hallucinated words and clauses in the generated text."""
    hard_labels = []
    for hallucinated_word in hallucinated_words:
        start_idx = generated_text.lower().find(hallucinated_word.lower())
        if start_idx != -1:
            end_idx = start_idx + len(hallucinated_word)
            hard_labels.append([start_idx, end_idx])
    for hallucinated_clause in hallucinated_clauses:
        clause = " ".join(hallucinated_clause.split()).replace(" ,", ",").replace(" .", ".")  # Ensure consistent spacing
        start_idx = generated_text.find(clause)
        if start_idx != -1:
            end_idx = start_idx + len(clause)
            hard_labels.append([start_idx, end_idx])
    return hard_labels

def apply_combined_logic(row):
    """Apply combined logic to handle single clause and multiple clauses."""
    clauses = split_into_clauses(row["model_output_text"])
    if len(clauses) == 1:
        updated_text = row["model_output_text"]
        hallucinated_clauses = []
        clause_similarity = []
    else:
        hallucinated_clauses, clause_similarity = find_hallucinated_clauses(row["model_output_text"], row["corrected_output"])
        updated_text = update_generated_text(row["model_output_text"], hallucinated_clauses)
    return updated_text, hallucinated_clauses, clause_similarity

column_names = ["model_input", "model_output_text", "corrected_output"]

# Create the DataFrame
df = pd.DataFrame({column_names[0]: model_inputs_list, column_names[1]: model_outputs_list, column_names[2]: answers})

df[["updated_generated_text", "hallucinated_clauses", "clause_similarity"]] = df.apply(
    lambda row: apply_combined_logic(row), axis=1, result_type="expand")

# Apply the functions to the DataFrame
df["hallucinated_words"] = df.apply(lambda row: extract_hallucinated_words(row["updated_generated_text"], row["corrected_output"]), axis=1)
df["hard_labels"] = df.apply(lambda row: get_hard_labels(row["model_output_text"], row["hallucinated_words"], row["hallucinated_clauses"]), axis=1)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
df.to_json('en_train_labeled.jsonl', orient="records", lines=True)